In [1]:
from __future__ import print_function
from __future__ import division

import pandas as pd
import glob
import os.path as op

In [32]:
def get_recruit_info(gcov):
    countfile = gcov.replace("genomecoverage", "aln_count")
    with open(countfile) as infile:
        recruit_count = infile.read().split()[1].strip()
    
    
    metagenome = op.basename(gcov).split("_vs_")[0].split("_")[0]
    sag = op.basename(gcov).split("_vs_")[1].split("_")[0]
    coverage = pd.read_csv(gcov, sep="\t", header=None)
    mean_per_contig = coverage.groupby([0])[2].mean() #.to_dict()
    sum_per_contig = coverage.groupby([0])[2].sum() #.to_dict()
    contig_size = coverage.groupby([0])[1].max()+1
    mean_sag_coverage = mean_per_contig.mean()
    totalbp = contig_size.sum()
    uncovered_bp = sum(coverage[2]==0)
    pct_covered = (totalbp - uncovered_bp)/totalbp * 100
    total_scaffold = len(sum_per_contig)
    uncovered_contig = sum(sum_per_contig==0)
    pct_scaffolds_covered = (total_scaffold - uncovered_contig)/total_scaffold *100
    sag = op.basename(gcov).split("_vs_")[1].strip(".genomecoverage")
    cols = ['method','sag','metagenome','Percent_scaffolds_with_any_coverage', 'Percent_of_reference_bases_covered', 'Average_coverage', 'Reads_recruited']
    data = ['bwa', sag, metagenome, 
           pct_scaffolds_covered,
           pct_covered, 
           mean_sag_coverage,
           recruit_count]
    df = pd.DataFrame(data, index=cols).transpose()
    return df
#    df = pd.DataFrame(data, columns=cols)
#    print(df)
    #   return df
    
#    df3 = DataFrame(np.random.randn(10, 5),
#...                 columns=['a', 'b', 'c', 'd', 'e'])

In [27]:
da = get_recruit_info(genomecov[0])

In [28]:
da

,sag,metagenome,Percent_scaffolds_with_any_coverage,Percent_of_reference_bases_covered,Average_coverage,Reads_recruited
0,AAA255A6_96912_95,2079,100,100,231.839,1397727


In [11]:
da.transpose()

,sag,metagenome,Percent_scaffolds_with_any_coverage,Percent_of_reference_bases_covered,Average_coverage
0,AAA255A6_96912_95,2079,100,100,231.839


In [17]:
cols = ['sag','metagenome','Percent_scaffolds_with_any_coverage', 'Percent_of_reference_bases_covered', 'Average_coverage', 'Reads_recruited']
big = pd.DataFrame(columns=cols)
big

,sag,metagenome,Percent_scaffolds_with_any_coverage,Percent_of_reference_bases_covered,Average_coverage,Reads_recruited


In [22]:
genomecov = glob.glob("../data/i454comp/*.genomecoverage")

cols = ['sag','metagenome','Percent_scaffolds_with_any_coverage', 'Percent_of_reference_bases_covered', 'Average_coverage', 'Reads_recruited']
big = pd.DataFrame(columns=cols)
for g in genomecov:
    new=get_recruit_info(g)
    big = pd.concat([big, new])

['../data/i454comp/2079_6_1746_gt150_fastq_extendedFrags_gt150_fastq_vs_AAA255A6_96912_95.genomecoverage',
 '../data/i454comp/2079_6_1746_gt150_fastq_extendedFrags_gt150_fastq_vs_AAA255B16_97682_95.genomecoverage',
 '../data/i454comp/2079_6_1746_gt150_fastq_extendedFrags_gt150_fastq_vs_AAA255C20_98068_95.genomecoverage',
 '../data/i454comp/GZFH_454_gt150_fastq_vs_AAA255A6_96912_95.genomecoverage',
 '../data/i454comp/GZFH_454_gt150_fastq_vs_AAA255B16_97682_95.genomecoverage',
 '../data/i454comp/GZFH_454_gt150_fastq_vs_AAA255C20_98068_95.genomecoverage']

In [34]:
big

,Average_coverage,Percent_of_reference_bases_covered,Percent_scaffolds_with_any_coverage,Reads_recruited,metagenome,method,sag
0,231.839,100,100,1397727,2079,bwa,AAA255A6_96912_95
0,579.587,100,100,2750421,2079,bwa,AAA255B16_97682_95
0,557.519,100,100,3402474,2079,bwa,AAA255C20_98068_95
0,2.65439,100,100,9492,GZFH,bwa,AAA255A6_96912_95
0,5.61372,100,100,17906,GZFH,bwa,AAA255B16_97682_95
0,5.41611,100,100,22245,GZFH,bwa,AAA255C20_98068_95


### Testing the latest version of the above function, which is returning funky results for percent_of_reference_bases_covered currently

In [2]:
def get_recruit_info(gcov):
    '''calculate information on recruited reads based on bedtools genomecoverage table

    Args:
        gcov (str): path to genome coverage file with recruitment pipeline naming convention of:
            metagenome_vs_sag.genomecoverage
            metagenome_vs_sag.aln_count file must also exists within the same directory

    Returns:
        pandas dataframe of genome coverage statistics
    '''
    countfile = gcov.replace("genomecoverage", "aln_count")
    with open(countfile) as infile:
        recruit_count = infile.read().split()[1].strip()

    metagenome = op.basename(gcov).split("_vs_")[0]
    sag = op.basename(gcov).split("_vs_")[1].split(".")[0]
    try:
        coverage = pd.read_csv(gcov, sep="\t", header=None)
        mean_per_contig = coverage.groupby([0])[2].mean()
        sum_per_contig = coverage.groupby([0])[2].sum()
        contig_size = coverage.groupby([0])[1].max() + 1
        mean_sag_coverage = mean_per_contig.mean()
        totalbp = contig_size.sum()
        # the sum of an evaluation?
        uncovered_bp = sum(coverage[2] == 0)
        pct_covered = ((totalbp - uncovered_bp) / totalbp) * 100
        total_scaffold = len(sum_per_contig)
        # the sum of an evaluation?
        uncovered_contig = sum(sum_per_contig == 0)
        pct_scaffolds_covered = ((total_scaffold - uncovered_contig) / total_scaffold) * 100
    # what are we excepting?
    except:
        mean_per_contig = 0
        sum_per_contig = 0
        contig_size = 0
        mean_sag_coverage = 0
        totalbp = 0
        uncovered_bp = 0
        pct_covered = 0
        total_scaffold = 0
        uncovered_contig = "NA"
        pct_scaffolds_covered = "NA" if uncovered_contig == 'NA' else (total_scaffold - uncovered_contig)/total_scaffold *100

    cols = ['sag',
            'metagenome',
            'Percent_scaffolds_with_any_coverage',
            'Percent_of_reference_bases_covered',
            'Average_coverage',
            'total_reads_recruited']
    data = [sag,
           metagenome,
           pct_scaffolds_covered,
           pct_covered,
           mean_sag_coverage,
           recruit_count]
    df = pd.DataFrame(data, index=cols).transpose()
    return df

In [3]:
genomecov = glob.glob("../data/i454comp/*.genomecoverage")
genomecov

['../data/i454comp/2079_6_1746_gt150_fastq_extendedFrags_gt150_fastq_vs_AAA255A6_96912_95.genomecoverage',
 '../data/i454comp/2079_6_1746_gt150_fastq_extendedFrags_gt150_fastq_vs_AAA255B16_97682_95.genomecoverage',
 '../data/i454comp/2079_6_1746_gt150_fastq_extendedFrags_gt150_fastq_vs_AAA255C20_98068_95.genomecoverage',
 '../data/i454comp/GZFH_454_gt150_fastq_vs_AAA255A6_96912_95.genomecoverage',
 '../data/i454comp/GZFH_454_gt150_fastq_vs_AAA255B16_97682_95.genomecoverage',
 '../data/i454comp/GZFH_454_gt150_fastq_vs_AAA255C20_98068_95.genomecoverage']

In [4]:
testdf = get_recruit_info(genomecov[1])

In [5]:
testdf

,sag,metagenome,Percent_scaffolds_with_any_coverage,Percent_of_reference_bases_covered,Average_coverage,total_reads_recruited
0,AAA255B16_97682_95,2079_6_1746_gt150_fastq_extendedFrags_gt150_fastq,100,100,579.587,2750421


In [15]:
gcov = genomecov[1]
coverage = pd.read_csv(gcov, sep="\t", header=None)
uncovered_bp = sum(coverage[2] == 0)
len(coverage[coverage[2] == 1])

56

In [14]:
gcov = genomecov[-1]
coverage = pd.read_csv(gcov, sep="\t", header=None)
uncovered_bp = sum(coverage[2] == 0)
len(coverage[coverage[2] == 1])

81356

In [13]:
!open ../data/i454comp/*.genomecoverage -a textedit

In [20]:
sum_per_contig = coverage.groupby([0])[2].sum()
sum_per_contig

uncovered_contig = len(sum_per_contig[sum_per_contig == 0])

In [21]:
uncovered_contig

0

In [23]:
sum_per_contig[sum_per_contig == 5]

Series([], Name: 2, dtype: int64)